**br_stat_profiler - Help Text**

In [1]:

import os
from user_args import load_parser, check_args

cmd = f"--help"
parser = load_parser()
args = parser.parse_args(cmd.split())
adict = check_args(args) 

usage: ipykernel_launcher.py [-h] [--version] [-i <GATKReport [stdin]>]
                             [-o <*.csv [stdout]>] [-lg <*.* [stderr]>]
                             [-mq <int min=1 [1]>] [-mxq <int max=100 [100]>]
                             [-eo <int 1 to 10 [4]>] [-sb <int 1 to 10 [4]>]
                             [-cb <int 1 to 15 [10]>] [-mic <int [1]>]
                             [-mxc <int [150]>] [-nan <float [None]>] [-sym]
                             [-z] [-ct <choices [cntxt]>] [-co <float [-20]>]
                             [-nW] [-wN <int [2]>] [-wRY <int [3]>]
                             [-wMSW <int [3]>] [-wBDHV <int [3]>]
                             [-V <choices [info]>] [-xRG]

br_stat_profiler (v1.2) - Converts GATK (V4.4.0.0) BaseRecalibrator stat
report into profiles that can be compared/clustered downstream. It generates a
separate profile for each ReadGroup in the stat report and tabulates them for
easy analysis. The profiles can be saved in a CSV fo

SystemExit: 0

/home/ido/miniconda3/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3513: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


**br_stat_profiling**<br>
Setting: qerr_cutoff  is turned off, 2 wobble_positions

In [1]:

import os

from user_args import load_parser, check_args, UARGS
from br_stat_profiler import preprocess_GATK_report, profile_rt

RECAL_TABLE_DIR = "./data/test_bqsr/"
GATK_FILE = "pre-LUAD-02_all_chrs_wo_Y_MT.bam.context4.recal_data.table"
# GATK_FILE = "HKNPC-101T.bam.GATKReport.mm_cntxt.6"
# GATK_FILE = "HKNPC-101T.bam.GATKReport.mm_cntxt.4"
REC_TAB_FULL_PATH = RECAL_TABLE_DIR + GATK_FILE
OUTFILE_DIR = "./outfiles/"
OUTFILE = OUTFILE_DIR + "first_profile_num.csv"
if os.path.exists(OUTFILE):
    os.remove(OUTFILE)

# cmd = f"--infile {REC_TAB_FULL_PATH} -o {OUTFILE} --extract_read_group --qerr_cutoff -20 --min_score 20 --scr_bin_count 3  -ct cyc"
cmd = f"--infile {REC_TAB_FULL_PATH} -o {OUTFILE} --extract_read_group --qerr_cutoff -20 --min_score 20 --scr_bin_count 3  -ct cntxt"
# cmd = f"--infile {REC_TAB_FULL_PATH} -o {OUTFILE} --extract_read_group --qerr_cutoff -20 --min_score 20 --scr_bin_count 3 -ct cntxt_cyc"
# cmd = f"--infile {REC_TAB_FULL_PATH} -o {OUTFILE} --extract_read_group --qerr_cutoff -20 --min_score 20 --scr_bin_count 3 -nW"

parser = load_parser()
args = parser.parse_args(cmd.split())
adict = check_args(args) 

rt2_pre_stat_df = preprocess_GATK_report(adict)
profile = profile_rt(rt2_pre_stat_df, adict)
with adict[UARGS.OUTFILE] as f:
    profile.to_csv(f)
        
print(profile.shape)
profile

0m:0s (  Init   ) - 2023-06-17 23:01:43,799 - INFO - 
 version                :	1.1
infile                 :	<_io.TextIOWrapper name='./data/test_bqsr/pre-LUAD-02_all_chrs_wo_Y_MT.bam.context4.recal_data.table' mode='r' encoding='UTF-8'>
outfile                :	<_io.TextIOWrapper name='./outfiles/first_profile_num.csv' mode='x' encoding='UTF-8'>
log_file               :	<ipykernel.iostream.OutStream object at 0x7f7b12c559c0>
min_score              :	20
max_score              :	100
min_err_observed       :	100
scr_bin_count          :	3
cyc_bin_count          :	10
min_cyc                :	1
max_cyc                :	150
nan_rep                :	None
qerr_cutoff_both_sides :	False
zscore                 :	False
cov_type               :	cntxt
qerr_cutoff            :	-20.0
no_wobble              :	False
max_wob_N_occ          :	2
max_wob_R_Y_occ        :	3
max_wob_M_S_W_occ      :	3
max_wob_B_D_H_V_occ    :	2
verbose                :	info
extract_read_group     :	True 
0m:0s (+0.0009 s) -

ddf_add_id_column


2m:6s (+0.7808 s) - 2023-06-17 23:03:50,785 - INFO - prepare_stat_df finished
2m:7s (+0.6199 s) - 2023-06-17 23:03:51,405 - INFO - br_stat_profiler finished!!!


(106350, 1)


,HVWKMCCXY
AAAA:0:Context,-2.30995
AAAA:1:Context,0.046981
AAAA:2:Context,6.033512
AAAB:0:Context,0.253638
AAAB:1:Context,2.28898
...,...
YYYV:1:Context,2.759209
YYYV:2:Context,6.383438
YYYW:0:Context,-0.207327
YYYW:1:Context,2.993084


#### **Step 1:** Binning Quality Scores into 4 bins (default value of scr_bin_count argument)

In [ ]:
rt2_pre_stat_df

#### **Step 2:** 
1) Calculating the Qerror (Empyrical Quality - Machine Quality) 
2) Calculating the Qerror of context with wobble positions
3) Completing the missing values as None or zero

In [ ]:
from br_stat_profiler import prepare_stat_df
from constants import RC_TAB2
cntxt_rt2_stat_df = prepare_stat_df(rt2_pre_stat_df, RC_TAB2.CNTXT_COV, adict)
cntxt_rt2_stat_df

#### QErrors Distribution

In [ ]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from constants import RT2_STAT
# Assuming you have a dataframe called 'df' with columns 'A', 'B', and 'C'
cutoff = 3
bins_count = 40
stat_df = cntxt_rt2_stat_df
data = stat_df[RT2_STAT.BIN_AVG_QLTY_ERR_COL]

# Plotting the histogram
counts, bins, patches = plt.hist(data, bins=bins_count)  # Adjust the number of bins as needed

for i in range(len(bins)-1):
    if bins[i] <= 0:
        patches[i].set_facecolor('tomato')
    else:
        patches[i].set_facecolor('cornflowerblue')

# plt.axvline(x=cutoff, color='black', linestyle='--',linewidth=1.5)
# plt.axvline(x=-cutoff, color='black', linestyle='--')

# Adding labels and title
plt.xlabel('Values')
plt.ylabel('Frequency')
plt.title('Histogram of Column B')

minx = math.floor(stat_df[RT2_STAT.BIN_AVG_QLTY_ERR_COL].min())
maxx =  math.ceil(stat_df[RT2_STAT.BIN_AVG_QLTY_ERR_COL].max())
xticks = np.arange(minx, maxx+1, 1)

plt.xticks(xticks, rotation=45)
# Display the histogram
plt.show()


NameError: name 'cntxt_rt2_stat_df' is not defined

#### QErrors Distribution - negative over positive values 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from constants import RT2_STAT
# Assuming you have a dataframe called 'df' with columns 'A', 'B', and 'C'
cutoff = 3
bins_count = 80
data = cntxt_rt2_stat_df[RT2_STAT.BIN_AVG_QLTY_ERR_COL]
# Generate example data
# data = np.random.normal(0, 1, 1000)

# Create the histogram for the positive bins
counts_pos, bins_pos, _ = plt.hist(data[data >= 0], bins=round(bins_count/2), color='cornflowerblue')

# Extract the negative bins and calculate the positive mirror image
negative_data = data[data < 0]
mirror_data = -negative_data

# Create the histogram for the mirror image of the negative bins
counts_mirror, bins_mirror, _ = plt.hist(mirror_data, bins=bins_pos, color='tomato')

plt.axvline(x=cutoff, color='black', linestyle='--', linewidth=1.5)

# Find the maximum count between positive and mirrored negative bins
max_count = max(np.max(counts_pos), np.max(counts_mirror))

# Set the y-axis limit to accommodate both positive and mirrored negative bins
plt.ylim(0, max_count)

# Display the plot
plt.show()


In [ ]:
na_count = cntxt_rt2_stat_df[RT2_STAT.BIN_AVG_QLTY_ERR_COL].isna().sum()
total = cntxt_rt2_stat_df.shape[0]
na_ratio = na_count/total
print(f"na couunt= {na_count}, total= {total}, na_ratio= {na_ratio}")
# NA are removed later in the profile extraction stage 


#### QError distribution by Score bins (covariate = Context)

In [ ]:
import numpy as np
import math

import matplotlib.pyplot as plt
import matplotlib.cm as cm
from constants import RC_TAB2,RT2_STAT, CNTXT_RT2, RANGES
from user_args import UARGS
# draw a dot chart using matplotlib

def score_qntl_ranges(rt2_pre_stat_df):
    df =  rt2_pre_stat_df.groupby(RC_TAB2.RG_SCORE_BIN_COL)[RC_TAB2.QLTY_SCORE_COL].apply(lambda x: (min(x), max(x))).reset_index()
    return df.rename(columns={RC_TAB2.QLTY_SCORE_COL: RANGES.SCORE_BIN})

def jitter_list(lst, mean=0, std=0.1):
    jittered_list = np.array(lst) + np.random.normal(mean, std, len(lst))
    return jittered_list.tolist()


temp_stat_df = cntxt_rt2_stat_df
temp_stat_df = temp_stat_df.sort_values(by=[RC_TAB2.CNTXT_COV])
y_start = math.floor(temp_stat_df[RT2_STAT.BIN_AVG_QLTY_ERR_COL].min())
y_end = math.ceil(temp_stat_df[RT2_STAT.BIN_AVG_QLTY_ERR_COL].max())
jitter_factor = (y_end - y_start) /100

ranges = score_qntl_ranges(rt2_pre_stat_df)[RANGES.SCORE_BIN].unique()
# cmap = cm.get_cmap('viridis', len(cntxt_rt2_stat_df[RC_TAB2.RG_SCORE_BIN_COL].unique()))
cmap = cm.get_cmap('viridis', adict[UARGS.SCORE_BINS_COUNT])
# create the scatter plot
fig, ax = plt.subplots()
for i, group in temp_stat_df.groupby(RC_TAB2.RG_SCORE_BIN_COL):
    jittered_y = jitter_list(group[RT2_STAT.BIN_AVG_QLTY_ERR_COL], 0 , jitter_factor)
    # ax.scatter(group[CNTXT_RT2.CNTXT_COL], group[RT2_STAT.QLTY_ERR_W_AVG_COL], 
    ax.scatter(group[CNTXT_RT2.CNTXT_COL], jittered_y, 
            #    c=[cmap(i)], label=i)
        c=[cmap(i)], label=f'{i} => {ranges[i][0]}-{ranges[i][1]}')

ax.axhline(y=0, color='black', linewidth=0.8)
ax.axhline(y=cutoff, color='red', linestyle='--', linewidth=1.5)
ax.set_title('Context vs Weighted QErrAvg')
ax.set_xlabel('Context')
ax.set_ylabel('Weighted QErrAvg (Phred)')
ax.legend(title="ScoreBins (0-3)", bbox_to_anchor=(1.05, 1), loc='upper left')

# plt.xticks(range(cyc_rt2_stat_df[CYC_RT2.CYC_BIN_COL].min(), cyc_rt2_stat_df[CYC_RT2.CYC_BIN_COL].max()+1,2))

import numpy as np
# Y_TICKS_COUNT = 5
# yticks = np.arange(y_start, y_end, (y_end - y_start) / (Y_TICKS_COUNT))
YTICK_STEP = 2
yticks = np.arange(y_start, y_end, YTICK_STEP)
yticks = np.append(yticks, 0)

plt.yticks(yticks)
# display the chart
plt.show()


In [ ]:
from br_stat_profiler import prepare_stat_df
from constants import RC_TAB2
mode = RC_TAB2.CYC_COV
cyc_rt2_stat_df = prepare_stat_df(rt2_pre_stat_df, mode, adict)
cyc_rt2_stat_df.head()

**QError Weighted Avg vs Read Cycle**

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from constants import RC_TAB2,RT2_STAT, CYC_RT2, RANGES
import matplotlib.colors as mcolors
from user_args import UARGS


def score_qntl_ranges(rt2_pre_stat_df):
    df =  rt2_pre_stat_df.groupby(RC_TAB2.RG_SCORE_BIN_COL)[RC_TAB2.QLTY_SCORE_COL].apply(lambda x: (min(x), max(x))).reset_index()
    return df.rename(columns={RC_TAB2.QLTY_SCORE_COL: RANGES.SCORE_BIN})

def jitter_list(lst, mean=0, std=0.1):
    jittered_list = np.array(lst) + np.random.normal(mean, std, len(lst))
    return jittered_list.tolist()

y_start = math.floor(cyc_rt2_stat_df[RT2_STAT.BIN_AVG_QLTY_ERR_COL].min())
y_end   = math.ceil(cyc_rt2_stat_df[RT2_STAT.BIN_AVG_QLTY_ERR_COL].max())
jitter_factor = (y_end - y_start) /100

ranges = score_qntl_ranges(rt2_pre_stat_df)[RANGES.SCORE_BIN].unique()
# cmap = cm.get_cmap('viridis', len(cyc_rt2_stat_df[RC_TAB2.RG_SCORE_BIN_COL].unique())+2)
cmap = cm.get_cmap('viridis', adict[UARGS.SCORE_BINS_COUNT])

fig, ax = plt.subplots()
for i, group in cyc_rt2_stat_df.groupby(RC_TAB2.RG_SCORE_BIN_COL):
    # print(i, cmap(i))
    jittered_y = jitter_list(group[RT2_STAT.BIN_AVG_QLTY_ERR_COL], 0 , jitter_factor)
    ax.scatter(group[CYC_RT2.CYC_BIN_COL], jittered_y,
        c=[cmap(i)], label=f'{i} => {ranges[i][0]}-{ranges[i][1]}')

ax.axhline(y=0, color='black', linewidth=.7)
ax.axhline(y=cutoff, color='red', linestyle='--', linewidth=1.5)
ax.set_title('CycleBin vs Weighted QError Avg.')
ax.set_xlabel('Read Cycle Bin')
ax.set_ylabel('Weighted QErrAvg (Phred)')
ax.legend(title="ScoreBins (0-3)", bbox_to_anchor=(1.05, 1), loc='upper left')

plt.xticks(range(cyc_rt2_stat_df[CYC_RT2.CYC_BIN_COL].min(), cyc_rt2_stat_df[CYC_RT2.CYC_BIN_COL].max()+1,2))
# plt.yticks(range(round(cyc_rt2_stat_df[RT2_STAT.QLTY_ERR_AVG_COL].min()), round(cyc_rt2_stat_df[RT2_STAT.QLTY_ERR_AVG_COL].max())+1))
# display the chart
plt.show()
